In [1]:
import torch
from torch import nn
from torchvision import models

from efficient_net.efficientdet.model import BiFPN
from efficient_net.efficientnet_pytorch.efficientnet import EfficientNet_Head
from efficient_net.efficientdet.utils import Anchors

In [3]:

class FTGen_1(nn.Module):
    def __init__(self, in_channels=64, out_channels=1):
        super(FTGen_1, self).__init__()
        self.ft = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, out_channels, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.ft(x)

class FourierBiFPN(nn.Module):
    def __init__(self, num_classes=3, compound_coef=0, load_weights=False, **kwargs):
        super(FourierBiFPN, self).__init__()
        
        self.compound_coef = compound_coef
        self.backbone_compound_coef = [0, 1, 2, 3, 4, 5, 6, 6]
        self.fpn_num_filters = [64, 88, 112, 160, 224, 288, 384, 384]
        self.fpn_cell_repeats = [3, 4, 5, 6, 7, 7, 8, 8]
        self.input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536]
        self.box_class_repeats = [3, 3, 3, 4, 4, 4, 5, 5]
        self.anchor_scale = [4., 4., 4., 4., 4., 4., 4., 5.]
        self.aspect_ratios = kwargs.get('ratios', [(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)])
        self.num_scales = len(kwargs.get('scales', [2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]))
        
        conv_channel_coef = {
            # the channels of P3/P4/P5.
            0: [40, 112, 320],
            1: [40, 112, 320],
            2: [48, 120, 352],
            3: [48, 136, 384],
            4: [56, 160, 448],
            5: [64, 176, 512],
            6: [72, 200, 576],
            7: [72, 200, 576],
        }

        num_anchors = len(self.aspect_ratios) * self.num_scales
        self.ftg1 = FTGen_1()
        self.upsample1 = nn.Upsample(size=(80, 80), mode="nearest")
        self.bifpn = nn.Sequential(
            *[BiFPN(self.fpn_num_filters[self.compound_coef],
                    conv_channel_coef[compound_coef],
                    True if _ == 0 else False,
                    attention=True if compound_coef < 6 else False)
              for _ in range(self.fpn_cell_repeats[compound_coef])])
        self.backbone_net = EfficientNet_Head(compound_coef=self.backbone_compound_coef[compound_coef])
    
    def forward(self, inputs):
        max_size = inputs.shape[-1]
        _, p3, p4, p5 = self.backbone_net(inputs)
        features = (p3, p4, p5)
        features = self.bifpn(features)
        
        p1,p2,p3,p4,p5 = features
        
        p3_f = nn.ReLU()(p3)
        p3_f = self.upsample1(p3_f)
        
        p4_f = nn.ReLU()(p4)
        p4_f = self.upsample1(p4_f)
        
        p5_f = nn.ReLU()(p5)
        p5_f = self.upsample1(p5_f)
        
        ft_3 = self.ftg1(p3_f)
        ft_4 = self.ftg1(p4_f)
        ft_5 = self.ftg1(p5_f)
        
        p3 = p3.reshape(p3.shape[0], -1)
        p4 = p4.reshape(p4.shape[0], -1)
        p5 = p5.reshape(p5.shape[0], -1)
        
        p3 = torch.sigmoid(p3)
        p4 = torch.sigmoid(p4)
        p5 = torch.sigmoid(p5)
        
        out = torch.cat((p3, p4, p5), 1).mean(dim=1)
        
        return out, ft_3, ft_4, ft_5

In [6]:
gpus = [0]

network = FourierBiFPN()


if torch.cuda.device_count() > 1:
    network = nn.DataParallel(network, device_ids=gpus).cuda(gpus[0])

for name, param in network.named_parameters():
    param.requires_grad = True

#         for param in network.module.backbone_net.parameters():
#              param.requires_grad = False

# loss definitions
criterion_bce = nn.BCELoss()
criterion_mse = nn.MSELoss()

Loaded pretrained weights for efficientnet-b0


In [9]:
import copy
import os
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

class Trainer(object):

    def __init__(self, network, optimizer, compute_loss, learning_rate=0.001, batch_size=32,
                 device='cpu', save_interval=2, save_path=''):

        self.network = network
        self.batch_size = batch_size
        self.optimizer = optimizer
        self.compute_loss = compute_loss
        self.device = device
        self.learning_rate = learning_rate
        self.save_interval = save_interval
        self.save_path = save_path
        self.writer = SummaryWriter('runs/bifpn-14')
        
#         if torch.cuda.device_count() > 1:
#             self.network = nn.DataParallel(self.network, device_ids=[0]).cuda()

    def load_model(self, model_filename):

        cp = torch.load(model_filename)
        self.network.load_state_dict(cp['state_dict'])
        start_epoch = cp['epoch']
        start_iter = cp['iteration']
        losses = cp['loss']
        return start_epoch, start_iter, losses

    def save_model(self, output_dir, epoch=0, iteration=0, losses=None, accuracy=None):
   
        saved_filename = 'model_{}_{}.pth'.format(epoch, iteration)
            
        saved_path = os.path.join(output_dir, saved_filename)
        cp = {'epoch': epoch,
              'iteration': iteration,
              'loss': losses,
              'state_dict': self.network.cpu().state_dict()
              }
        torch.save(cp, saved_path)
        self.network.to(self.device)

    def train(self, dataloader, n_epochs=25, output_dir='out', model=None, freeze=False):
    

        start_epoch = 0
        start_iter = 0
        losses = []
        print('TRAINING')

        # setup optimizer
        self.network.train(True)
        best_loss = float("inf")
        best_model_wts = copy.deepcopy(self.network.state_dict())
        best_acc = 0.0
        # STARTING TRAINING LOOP
        for epoch in range(start_epoch, n_epochs):
            if (epoch+1) == 2:
                for param in self.network.module.backbone_net.parameters():
                    param.requires_grad = True
            train_loss_history = []
            val_loss_history = []
            train_acc_history = []
            
            val_acc_history = []
            for phase in ['train', 'val']:	
                # Set model to training mode
                if phase == 'train':
                    self.network.train()
                # Set model to evaluate mode
                else:
                    self.network.eval()
                #print(type(dataloader[phase]))
                for i, data in enumerate(dataloader[phase], 0):
                    
                    if i >= start_iter:
                        start = time.time()
                        img, labels, ft_feat = data
                        self.optimizer.zero_grad()
                        with torch.set_grad_enabled(phase == 'train'):
                            loss, acc, = self.compute_loss(
                                self.network, img, labels, ft_feat, self.writer, phase, self.device)
                            if phase == 'train':
                                loss.backward()
                                self.optimizer.step()
                                train_loss_history.append(loss.item())
                                train_acc_history.append(acc)
                            else:
                                val_loss_history.append(loss.item())
                                val_acc_history.append(acc)
                        end = time.time()

                        print(
                            f"[{epoch}/{n_epochs}][{i}/{len(dataloader[phase])}] => LOSS: {loss.item()}, ACC: {acc}, (ELAPSED TIME: {(end - start)}), PHASE: {phase}")
                        losses.append(loss.item())
            epoch_train_loss = np.mean(train_loss_history)
            epoch_train_acc = np.mean(train_acc_history)
            print(f"TRAIN LOSS: {epoch_train_loss} TRAIN ACC: {epoch_train_acc}  EPOCH: {epoch}")

            epoch_val_loss = np.mean(val_loss_history)
            epoch_val_acc = np.mean(val_acc_history)
                
            print(f"VAL LOSS: {epoch_val_loss}, EPOCH: {epoch}, ACC: {epoch_val_acc}")

            if phase == 'val' and epoch_val_acc > best_acc:
                print(
                    f"VAL ACC IMPROVED FROM: {epoch_val_acc} TO: {best_acc}, COPYING OVER NEW SWEIGHTS")
                best_acc = epoch_val_acc
                best_model_wts = copy.deepcopy(self.network.state_dict())
                    
            print(f"EPOCH {epoch + 1} DONE")

            # save the last model, and the ones in the specified interval
            if (epoch + 1) == n_epochs or epoch % self.save_interval == 0:
                self.save_model(output_dir, epoch=(epoch + 1),
                                iteration=0, losses=losses, accuracy=int(epoch_val_acc*100))

        self.network.load_state_dict(best_model_wts)
        torch.save(self.network, self.save_path)
        self.save_model(output_dir, epoch=0, iteration=0, losses=losses)
        self.writer.close()

In [10]:
import torch.optim as optim

learning_rate = 0.001
weight_decay = 0.00001
optimizer = optim.Adam(filter(lambda p: p.requires_grad, network.parameters()), 
                        lr=learning_rate, weight_decay=weight_decay)


In [11]:
from torch.autograd import Variable

def compute_loss(network, img, labels, ft_feat, logger, phase, device):
    """
    Compute the losses, given the network, data and labels and 
    device in which the computation will be performed. 
    """
 
#     imagesv = Variable(img['image'].to(device))
#     labelsv_pixel = Variable(labels['pixel_mask'].to(device))
    
    threshold = 0.5
    
    labelsv_binary = Variable(labels.to(device))
    ft_feat = Variable(ft_feat.to(device))
    
    out, ft_3, ft_4, ft_5 = network(img.to(device))
    
    preds = (out > threshold)
    acc = float((preds == labelsv_binary).sum())/float(out.shape[0])
    
    loss1 = criterion_bce(out.unsqueeze(1), labelsv_binary.unsqueeze(1).float())
    loss2 = criterion_mse(ft_3, ft_feat)
    loss3 = criterion_mse(ft_4, ft_feat)
    loss4 = criterion_mse(ft_5, ft_feat)
    
    
    loss_temp = (loss2 + loss3 + loss4)/3.0
    
    loss = 0.5 * loss1 + 0.5 * (loss_temp)
    return loss, acc


In [12]:
trainer = Trainer(network, optimizer, compute_loss, learning_rate=learning_rate,
                      batch_size=32, device=f'cuda:{gpus[0]}' if torch.cuda.is_available() else 'cpu')
